# Import Libray

In [4]:
from live_connection import Live_connection
import threading
import torch
from PIL import Image
from io import BytesIO
import time
import numpy as np

# Setup stuffs

## Load and setup model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6', pretrained=True)
model.classes = [0,59,63,67]# person, bed, laptop(as some phone can be detact by laptop), cell phone 
torch.set_num_interop_threads(8)# improve performance
torch.set_num_threads(8)

## Start and Setup socket(Connect camera)

In [ ]:
live_connection = Live_connection(input("Address to bind:"),7777)# 7777 is the default port

live_connection.start_connection()

t1 = threading.Thread(live_connection.start_recieve())
t1.start()

## Variable of current state

In [ ]:
## lying bed
lyingonbed = False
startlyingtime = 0
notlyingtimestart = 0
chancelying = 3
## touching phone
Touchingphone = False
starttouchingtime = 0
nottouchingtimestart = 0
chancetouching = 3
## sleeping
sleeping = False
startsleepingtime = 0
notsleepingtimestart = 0
## temperatures
tempPerson = 0
### core temp requires face recongnition
tempBed = 0

# Condition for automation

## Person detection

In [ ]:
def isPersonAvailable(pandas)-> bool:
    return 0 in pandas['class'].array

## Bed lying detection

In [ ]:
def isLyinOnBed(pandas,k:int)->bool:
    #pandasDataFrame is the output of model(pandas.DataFrame)
    #return true if the person is lying on the bed
    #return false if the person is not lying on the bed

    # Check if the person is on bed     
    
    # Calculate the length ratio of person on the bed
    personxlen = pandas[pandas['class'] == 0]['xmax'].array[k] - pandas[pandas['class'] == 0]['xmin'].array[k]
    personylen = pandas[pandas['class'] == 0]['ymax'].array[k] - pandas[pandas['class'] == 0]['ymin'].array[k]
    if not(59 in pandas['class'].array):
        if personylen>personxlen:
            if(personylen/personxlen>3.7 and personylen/personxlen<4.3):# acoording to web search, the ratio of length of shoulder is 4:1
                return True
            else:
                return False
    bedxlen = pandas[pandas['class'] == 59]['xmax'].array[0] - pandas[pandas['class'] == 59]['xmin'].array[0]
    bedylen = pandas[pandas['class'] == 59]['ymax'].array[0] - pandas[pandas['class'] == 59]['ymin'].array[0]

    if bedxlen > bedylen :
            return personxlen/bedxlen > 0.7
    else:
            return personylen/bedylen > 0.7


## Touching phone detection

In [3]:
def isTouchingPhone(pandas,person)->bool:
    #pandasDataFrame is the output of model(pandas.DataFrame)
    #return true if the person is touching the phone
    #return false if the person is not touching the phone
    #return None if the person is not in the frame
    # Check if the person is on bed
    phone = []
    if not(63 in pandas['class'].array):
        return False
    else:
        phone.append(pandas[pandas['class'] == 63])
    if not(67 in pandas['class'].array):
        return False
    else:
        phone.append(pandas[pandas['class'] == 67])
    person = pandas[pandas['class'] == 0]
    # see if the person touches the phone
    overlapArea = 0
    for x in person:
        for y in phone:
            for z in range(len(y)):
                x_overlap = Math.max(0, Math.min(y['xmax'].array[z], person['xmax'].array[x]) - Math.max(y['xmin'].array[z], person['xmin'].array[x]));
                y_overlap = Math.max(0, Math.min(y['ymax'].array[z], person['ymax'].array[x]) - Math.max(y['ymin'].array[z], person['ymin'].array[x]));
                overlapArea = Math.Max(overlapArea,x_overlap * y_overlap);
                areaphone = (y['xmax'].array[z] - y['xmin'].array[z])*(y['ymax'].array[z] - y['ymin'].array[z])
                if overlapArea/areaphone > 0.5:
                    return True
    return False

## Bed Temperature

In [ ]:
def bedtemperature(pandas,thermal,person):
    ## calculate the temperature of the bed by calculate the average temperature of the bed exclude the person
    ## return the temperature of the bed
    temp = 0
    count = 0
    for x in range(len(thermal[0])):
        for y in range(len(thermal)):
            if ( pandas[pandas['class'] == 59]['xmax'] <= x and np.all(x > pandas[pandas['class'] == 0]['xmax'].array[person] )     \
            or   pandas[pandas['class'] == 0]['xmin'] >= x and np.all(x < pandas[pandas['class'] == 0]['xmin'].array[person]  )  )  \
            and                                                                                                                     \
               ( pandas[pandas['class'] == 59]['ymax'] <= x and np.all(x > pandas[pandas['class'] == 0]['ymax'].array[person] )     \
            or   pandas[pandas['class'] == 0]['ymin'] >= x and np.all(x < pandas[pandas['class'] == 0]['ymin'].array[person]  )  ):
                count+=1
                temp = temp + (thermal[x][y] - temp)/count
    return temp 

## Body Temperature

In [ ]:
def bodytemperture(pandas,thermal,k:int):
    ## calculate the temperature of the person by calculate the average temperature of the person
    ## return the temperature of the person
    ## reduce the insignificant temperature(extermely low or high,ie >40, <20)
    temp = 0
    count = 0
    for x in range(len(thermal[0])):
        for y in range(len(thermal)):
            ## check if the point is in the person
            if ( pandas[pandas['class'] == 0]['xmax'].array[k] <= x and np.all(x >= pandas[pandas['class'] == 0]['xmin'].array[k] ) \
            and                                                                                                                     \
                pandas[pandas['class'] == 0]['ymax'].array[k] <= x and np.all(x >= pandas[pandas['class'] == 0]['ymin'].array[k] ) )\
            and thermal[x][y] < 40 and thermal[x][y] > 20:
                 count+=1
                 temp = temp + (thermal[x][y] - temp)/count
    return temp

## Detection Main

In [ ]:
def detections(pandas,thermal,timenow):
    noperson = len(pandas[pandas['class'] == 0])
    if(isPersonAvailable(pandas)):
            pass # can add time automation

    ## lying on bed
    re = False
    person = []
    for k in range(noperson):
        if(isLyinOnBed(pandas,k)):
            re = True
            person.append(k)
    if re:
        chancelying = 0
        if not lyingonbed:
            lyingonbed = True
            startlyingtime = timenow
    else:
        if lyingonbed:
            chancelying += 1
            if chancelying > 3:
                notlyingtimestart = timenow
                lyingonbed = False
                print("Lying on bed for",timenow-startlyingtime,"seconds")
                startlyingtime = 0
    
    ## touching phone
    if isTouchingPhone(pandas,person):
        chanceTouching = 0
        if not Touchingphone:
            Touchingphone = True
            starttouchingtime = timenow
    else:
        if Touchingphone:
            chanceTouching += 1
            if chanceTouching > 3:
                nottouchingtimestart = timenow
                Touchingphone = False
                print("Touching phone for",timenow-starttouchingtime,"seconds")
                starttouchingtime = 0

    ### sleeping or in sleep mood
    # if lyingfor bed for 10 seconds and not touching phone for 10 seconds, then sleep     
    if lyingonbed and not Touchingphone:
        if not sleeping:
            startsleepingtime = timenow
            sleeping = True
    else:
        if sleeping:
            print("Sleeping for",timenow-startsleepingtime,"seconds")
            startsleepingtime = 0
            sleeping = False
    ## calculate the temperature of the bed
    tempBed = bedtemperature(pandas,thermal,person)
    if lyingonbed:
        temmpbody = bodytemperture(pandas,thermal,person)
            

# Actions

> All action are customizable, are highly extensible

In [ ]:
#dummy variable for example
playingmusic = False 
lights = False 
aircon = False
aircontemp = 25

## When lying on bed

In [ ]:
def OnLyingBed(lying:bool,startlyingtime:float,stoplyingtime:float):
    # Here are some examples
    ## if the person is lying on the bed for more than 5 seconds, play music
    if startlyingtime-time.time() >5 and not playingmusic:
        #play music
        playmusic = True
    elif stoplyingtime-time.time() >5 and playingmusic:
        #stop music
        playmusic = False
    ## air con control
    

## When using(touching) phone

In [ ]:
def OnTouchingPhone(touching:bool,starttouchingtime:float,stoptouchingtime:float):
    # Here are some examples
    ## if the person is touching the phone for more than 15 seconds, turn on the lights
    if starttouchingtime-time.time() >15 and not lights:
        #turn on lights
        lights = True

## When sleeping(on sleep mood)

In [ ]:
def OnSleeping(sleeping:bool,startsleepingtime:float,stopsleepingtime:float):
    # Here are some examples
    ## if the person is in sleep mood for 5 seconds, turn off the lights
    if startsleepingtime-time.time() >5 and lights:
        #turn off lights
        lights = False
        playmusic = False
    ## air con control

## Add more if you like

In [ ]:
## Action main
def actions():
    t1 = threading.Thread(target=OnLyingBed,args=(lyingonbed,startlyingtime,notlyingtimestart))
    t2 = threading.Thread(target=OnTouchingPhone,args=(Touchingphone,starttouchingtime,nottouchingtimestart))
    t3 = threading.Thread(target=OnSleeping,args=(sleeping,startsleepingtime,notsleepingtimestart))
    ## Add more if you like
    t1.start()
    t2.start()
    t3.start()
    

# Main

In [ ]:
while True:
    #get data
    if live_connection.new_frame_avaliable:
        (frame,thermaldata)= live_connection.getcurrentframe()
        
        ## get into threading
        t2 = threading.Thread(target=worker,args=(frame,thermaldata,thermaldata))
        t2.start()



    def worker(frame,thermaldata):
        ## convert bytes to image
        timenow = time.time()
        image = Image.open(BytesIO(frame))
        
        ## make prediction
        results = model(image)
        results.show()
        
        ## get the result
        pandas = results.pandas().xyxy[0]
        # DETECTIONS
        detections(pandas,timenow)
        # ACTIONS
        
        